In [1]:
import torch

# Scalar example
x = torch.tensor(2.0, requires_grad=True)
y = x**2 + 3*x + 1   # y = x² + 3x + 1

y.backward()  # dy/dx

print("x =", x.item())
print("y =", y.item())
print("dy/dx =", x.grad.item())  # Should be 2x + 3 = 7


x = 2.0
y = 11.0
dy/dx = 7.0


In [2]:
x = torch.randn(3, requires_grad=True)  # vector
A = torch.tensor([[2., 1., 0.],
                  [0., 3., 1.]], requires_grad=False)

y = (A @ x).sum()  # scalar output

y.backward()
print("x =", x)
print("Gradient dy/dx =", x.grad)


x = tensor([0.8707, 0.9710, 0.6764], requires_grad=True)
Gradient dy/dx = tensor([2., 4., 1.])


In [3]:
x = torch.tensor([1.0, 2.0], requires_grad=True)

y = torch.stack([x[0]**2, x[0]*x[1], torch.sin(x[1])])

# Compute full Jacobian
J = torch.autograd.functional.jacobian(lambda v: torch.stack([v[0]**2, v[0]*v[1], torch.sin(v[1])]), x)

print("Jacobian:\n", J)


Jacobian:
 tensor([[ 2.0000,  0.0000],
        [ 2.0000,  1.0000],
        [ 0.0000, -0.4161]])


In [4]:
import torch
torch.set_default_dtype(torch.float64)

def nth_derivative(f, x, n):
    """
    f: function mapping scalar tensor -> scalar tensor
    x: scalar tensor with requires_grad=True
    n: order (>=1)
    """
    y = f(x)
    for k in range(n):
        # Always create_graph to enable higher-order grads
        (y,) = torch.autograd.grad(y, x, create_graph=True)
    return y  # n-th derivative evaluated at x

# Example: f(x) = sin(x^3); compute d^5 f / dx^5 at x = 1.0
x = torch.tensor(1.0, requires_grad=True)
f = lambda t: torch.sin(t**3)
d5 = nth_derivative(f, x, 5)
print("5th derivative at x=1:", d5.item())


5th derivative at x=1: 24.493920508725523


In [5]:
import torch
from torch.autograd.functional import hessian

# f: R^2 -> R
def f(v):
    x, y = v
    return x**3 * y**2 + torch.sin(x*y)

# Point
v = torch.tensor([1.0, 2.0], requires_grad=True)

H = hessian(lambda z: f(z), v)
print("Hessian shape:", H.shape)   # (2, 2)
print(H)


Hessian shape: torch.Size([2, 2])
tensor([[20.3628,  9.7653],
        [ 9.7653,  1.0907]])


In [6]:
x = torch.tensor(1.0, requires_grad=True)
y = torch.tensor(2.0, requires_grad=True)

def fxy(x, y):
    return x**3 * y**2 + torch.sin(x*y)

# First w.r.t. x
g_x, = torch.autograd.grad(fxy(x, y), x, create_graph=True)
# Then w.r.t. y
g_xy, = torch.autograd.grad(g_x, y, create_graph=True)
# Then w.r.t. x again
g_xyx, = torch.autograd.grad(g_xy, x, create_graph=True)

print("∂³f/∂x∂y∂x at (1,2):", g_xyx.item())


∂³f/∂x∂y∂x at (1,2): 22.027397638885844
